# Music Note Classifier
### The goal of this program is to create a useful segment of code to convert mp3 files into usable data for a machine learning model.

## Import dependencies:

The four imports used to achieve this functionality are numpy, pandas, os, and librosa. Librosa used in conjunction with numpy allows for easy spectogram generation and feature extraction. The os and csv are used for I/O and saving the features extracted to a csv file to push to the machine learning model.

In [1]:
import numpy as np
import pandas as pd
import os
import librosa
from librosa import feature
import csv

## Feature Extraction and Spectrogram Functions:
In this section two functions are defined: load_spec and extract features. The load spec function loads the spectogram, trims it, and returns the mean of it. The extract_features takes in the spectogram and calls a list of feature extraction functions, takes the mean of them, and stores them into an array.

In [2]:
def load_spec(filename):
    y, sr = librosa.load(filename)
    y_trimmed, _ = librosa.effects.trim(y)
    D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
    D = np.mean(D)
#     mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
#     S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
#     S_db_mel = librosa.amplitude_to_db(S, ref=np.max)
    return D

In [3]:
# fn_list_ii = [
#  feature.rmse,
#  feature.zero_crossing_rate
# ]

def extract_features(y,sr):
    chroma = np.mean(feature.chroma_stft(y=y, sr=sr))
    centr = np.mean(feature.spectral_centroid(y=y, sr=sr))
    band = np.mean(feature.spectral_bandwidth(y=y, sr=sr))
    roll = np.mean(feature.spectral_rolloff(y=y, sr=sr))
    
    feat_vect = [chroma, centr, band, roll] 
    return feat_vect

## Data Feature Extraction Loop:
This section loops through all of the files and runs the extraction functions on them, storing the results into a notes features array which will be used later to create the csv file for this project. A label is also created for each folder of different notes for the y value of each row in the csv file.

In [4]:
count = 0
notes_feats = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    count +=1
    if(count > 3):
        i = dirname.rfind('/')
        label = dirname[i+1:]
        print(label)
    for filename in filenames:
#         d = load_spec(os.path.join(dirname, filename))
#         pair = (label, d)
#         list1.append(pair)
        
        y, sr = librosa.load(os.path.join(dirname, filename),sr=None)
        feat_vect = extract_features(y,sr)
        feat_vect.insert(0, label)
        notes_feats.append(feat_vect)

23. D1
34. C-sharp 2
37. E3
26. F2
14. F1
5. G-sharp
17. G-sharp 1
31. A-sharp 2
24. D-sharp 1
11. D
21. C1
10. C-sharp
20. B1
36. D-sharp 2
8. B


/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


15. F-sharp 1
33. C2
19. A-sharp 1
16. G1
25. E2
4. G
7. A-sharp
18. A1
35. D2
9. C
6. A
29. G-sharp 2
22. C-sharp 1
28. G2
1. E
2. F
27. F-sharp 2
13. E1
32. B2
30. A2
12. D-sharp
3. F-sharp


## CSV File Creation:
In this section the csv is populated and stored into a file called Guitar_features.csv. First the headers for each column are defined and then the rest of the rows are filed in with the 2d array stored in notes_feats.

In [5]:
guitar_output = 'Guitar_features.csv'

header =[
    'note_names',
    'chroma_stft',
    'spectral_centroid',
    'spectral_bandwidth',
    'spectral_rolloff'
#     'rmse',
#     'zero_crossing_rate'
]

with open(guitar_output,'+w') as f:
    csv_writer = csv.writer(f, delimiter = ',')
    csv_writer.writerow(header)
    csv_writer.writerows(notes_feats)